In [11]:
import pandas as pd
import numpy as np
#from tqdm import tnrange, tqdm_notebook
#from time import sleep
import csv
import sys


In [12]:
modelo = pd.read_csv('Sim01.txt', sep ='\s+', header = 0) #se debe ingresar el nombre del modelo.

In [13]:
print modelo

             XC      YC      ZC    FE  RT   DEN TON
0        8005.0  1555.0   765.0  -1.0  -1  -1.0  -1
1        8015.0  1555.0   765.0  -1.0  -1  -1.0  -1
2        8025.0  1555.0   765.0  -1.0  -1  -1.0  -1
3        8035.0  1555.0   765.0  -1.0  -1  -1.0  -1
4        8045.0  1555.0   765.0  -1.0  -1  -1.0  -1
5        8055.0  1555.0   765.0  -1.0  -1  -1.0  -1
6        8065.0  1555.0   765.0  -1.0  -1  -1.0  -1
7        8075.0  1555.0   765.0  -1.0  -1  -1.0  -1
8        8085.0  1555.0   765.0  -1.0  -1  -1.0  -1
9        8095.0  1555.0   765.0  -1.0  -1  -1.0  -1
10       8105.0  1555.0   765.0  -1.0  -1  -1.0  -1
11       8115.0  1555.0   765.0  -1.0  -1  -1.0  -1
12       8125.0  1555.0   765.0  -1.0  -1  -1.0  -1
13       8135.0  1555.0   765.0  -1.0  -1  -1.0  -1
14       8145.0  1555.0   765.0  -1.0  -1  -1.0  -1
15       8155.0  1555.0   765.0  -1.0  -1  -1.0  -1
16       8165.0  1555.0   765.0  -1.0  -1  -1.0  -1
17       8175.0  1555.0   765.0  -1.0  -1  -1.0  -1
18       818

In [6]:
bx=modelo.ix[:,0]
by=modelo.ix[:,1]
bz=modelo.ix[:,2]
ton= modelo.ix[:,5] #indicar columna con valores de tonelaje

In [7]:
xmin= min(bx)
ymin= min(by)
zmin= min(bz)
xmax = max(bx)
ymax = max(by)
zmax =max (bz)
sx=10
sy=10
sz=10

In [8]:
bxy= {}
for i in range(len(bx)):
    cordx = int(bx.iloc[i])
    cordy = int(by.iloc[i])
    if ("("+str(cordx)+","+str(cordy)+")" in bxy):
        bxy["("+str(cordx)+","+str(cordy)+")"].append(i)
    else:
        bxy["("+str(cordx)+","+str(cordy)+")"]=[i]

print "\t Columnas indentificadas (xy)"

	 Columnas indentificadas (xy)


In [9]:
#####Input
project_folder = r"C:\VULCAN_HOME\AnacondaProjects\footprint\estadisticas_4"
c_mina = 4.1
c_planta = 8.1
c_venta = 0.3
rec = 0.85
precio = 2.6
r = 0.1
dens = 2.6
max_height = 300
tpd = 200
util = 200
area = 225
dev_cost = 0 #2200
f =2204.62
sim = 100 #indique cuantas simulaciones contiene el modelo de bloques
in_sim=  7 #indique la columna en la cual comienza los atributos de simulaciones en el modelo
vmr=(tpd*util)/(dens*area) #este valor equivale a los metros extraidos en un año
dblock=sz/vmr           #Corresponde al porcentaje de vida de un bloque en un nivel por año
niveles= int((zmax-zmin)/sz)

In [16]:
#Tasa de descuento
desc = []
dsc = 0
aux = 0

#Secuencia de bloques
for i in range(1, niveles+1):
    aux = aux + dblock
    if (i*sz >= max_height):
        dsc=0
    else:
        dsc = 1/((1+r)**aux)
    desc.append(dsc)
    
print "\t Tasa de descuento aplicado por bloque, creado"

	 Tasa de descuento aplicado por bloque, creado


In [12]:
#valorizar cada simulacion 
rows = len(bx)
columns = len(range(6, sim+6)) #(range(6,sim+6))
m_val = np.empty((rows, 1)) #Columns

#for j in tqdm_notebook(range(6,sim+6), desc="\t Value"): #range(6,sim+6)
for i in range(len(bx)):
    ley_cu =  modelo.ix[i,3]/100
    bton = ton.ix[i]
    bval = ((precio-c_venta)*rec*f*bton*ley_cu)-(c_mina+c_planta)*bton
    m_val[i][0]= bval  # m_val[i][j-6]


In [20]:
#valorizacion acumulada por cada xy
m_val_acum = np.zeros((rows, columns))
for m in tqdm_notebook(range(sim),desc='sim'): #sim
      #### val_acum = 0 
    for i in bxy.keys():     #in bxy.keys()
        val_acum = 0
        for k in range(1, niveles+1):
            for l in bxy[i]:
                piso=int(abs(bz.ix[l]-zmin)/sz)
                if (piso>=k):
                    vdesc = m_val[l][m]*desc[piso-k] #...editado..val[l][m] #####verificar esta variable!!!!!!!!!!
                    val_acum = val_acum + vdesc  #Los valores que estan sobre el HOD tienen vdesc=0
                    m_val_acum[l][m] = val_acum
                                    

                
            
                

In [26]:
#guardar informacion footprint
m_foot = np.zeros((rows, columns))
m_columna = np.zeros((rows, columns))
for m in tqdm_notebook(range(sim),desc='sim'):    
    #for q in range(1,niveles+1):
    #nc = 0
    
        #ncp = 0
    for i in bxy.keys():
        #nc = 0
        temp_1 = 0
        for l in bxy[i]:
            if (m_val_acum[l][m]>(dev_cost*area)):  #genero una matriz con los valores acumulados en cada bloque mas la cantidad de bloques de columna
                temp_1 = m_val_acum[l][m]-dev_cost*area
                m_foot[l][m] = temp_1
                

In [22]:
m_foot = np.zeros((rows, 1)) #columns
m_val_acum = np.zeros((rows, 1))
m_columna = np.zeros((rows, 1))
#for m in tqdm_notebook(range(sim),desc='sim'): #sim
vxy= {}
tonxy = {}
ton_acum = 0
val_acum = 0
for i in bxy.keys():     #in bxy.keys()
    for k in range(1, niveles+1):
        for l in bxy[i]:
            piso=int(abs(bz.ix[l]-zmin)/sz) # se suma +1, ya que parte del nivel 1 y no 0 ?????
            if (piso>=k):# and (ton.ix[l]>0) and (k<(piso+(max_height/sz)))    #genero un control en el tonelaje acumulado sobre la frontera de HOD.
                vdesc = m_val[l][0]*desc[piso-k] #...editado..val[l][m] ...vdesc = m_val[l][m]*desc[piso-k]
                val_acum = val_acum + vdesc  #Los valores que estan sobre el HOD tienen vdesc=0
                m_val_acum[l][0] = val_acum #m
                if (k<(piso+(max_height/sz))):
                    ton_acum = ton_acum + ton.ix[l] #se suman solo los bloques que tienen ton.
                else:
                    ton_acum = 0
        if (i in vxy):
            vxy[i].append(val_acum)
            tonxy[i].append(ton_acum)
            val_acum = 0
            ton_acum = 0
        else:
            vxy[i] = [val_acum]
            tonxy[i] = [ton_acum]
            val_acum = 0
            ton_acum = 0

                               
NCP = {}
UCL = {}
Area_foot = {}
UCLton = {}
    
    
    # "agregar una columna ff al modelo"
for q in range(1,niveles+1):  #se ve en cada piso los valores positivos y se suman.
    suma = 0
    suma2 = 0
    ncp = 0
    area_foot = 0
    for i in vxy.keys():
        nc = 0
        temp_1 = 0
        for l in bxy[i]:
            if (vxy[i][q-1]>dev_cost*area): # and (ton.ix[l]>0) and (k<(piso+(max_height/sz))) 
                temp = vxy[i][q-1]-dev_cost*area
                temp2 = tonxy[i][q-1]
                suma = suma+temp
                suma2 = suma2 + temp2
                area_foot = area_foot + sx*sy
                ncp=ncp+1
                if (m_val_acum[l][0]>dev_cost*area): #m #genero una matriz con los valores acumulados en cada bloque mas la cantidad de bloques de columna
                    temp_1= m_val_acum[l][0]-dev_cost*area #m
                    m_foot[l][0]= temp_1 #m
                    
                         
    if (q in UCL):
        UCL[q].apppend(suma)
        NCP[q].append(ncp)
        UCLton[q].append(suma2)
        Area_foot[q]=[area_foot]
        
    else:
        UCL[q]=[suma]
        NCP[q]=[ncp]
        UCLton[q]= [suma2]
        Area_foot[q]=[area_foot]
                
vpiso=open(project_folder+'\_Footprint_modelo_sim_costos_kcu'+str(1)+'.xls','w')
vpiso.write("nivel"+"\tZ"+"\tTons"+"\tValor_acum_col"+"\tArea")
for s in UCL.keys(): # fijarme que esta i no se intercepte con las de arriba
    vpiso.write("\n"+str(s))
    vpiso.write("\t"+str(zmin+(s-1)*sz))
    vpiso.write("\t"+str(UCLton[s][0]))
    vpiso.write("\t"+str(UCL[s][0]))
    vpiso.write("\t"+str(Area_foot[s][0]))
vpiso.close()
            #print "\t Suma acumulada por columna, de la sim"+str(m)+" calculada :D"
            
      
                
                    
                


In [27]:
modelo_footprint = pd.DataFrame(m_foot)
modelo_columna = pd.DataFrame(m_columna)

In [ ]:
m_valorizado = pd.DataFrame(m_val)


In [23]:
m_acum = pd.DataFrame(m_val_acum)

In [24]:
m_acum.to_csv('modelo_acumulado_f.csv', sep=' ', header=None)

In [28]:
modelo_footprint.to_csv('modelo_footprint.csv', sep=' ', header=None)
modelo_columna.to_csv('modelo_columna.csv', sep = ' ', header = None)

In [20]:
m_valorizado.to_csv('modelo_valorizado.csv', sep = ' ', header=None)

In [ ]:
""""
NCP = {}
UCL = {}
Area_foot = {}
UCLton = {}
m_foot = np.zeros((rows, columns))
m_columna = np.zeros((rows, columns))
# "agregar una columna ff al modelo"
for q in tqdm_notebook(range(1,niveles+1),desc= "\t footprint"):  #se ve en cada piso los valores positivos y se suman.
    suma = 0
    suma2 = 0
    ncp = 0
    area_foot = 0
    for i in vxy.keys():
        nc = 0
        suma_1 = 0
        for l in bxy[i]:
            if (vxy[i][q-1]>dev_cost*area): # and (ton.ix[l]>0) and (k<(piso+(max_height/sz))) 
                temp = vxy[i][q-1]-dev_cost*area
                temp2 = tonxy[i][q-1]
                suma = suma+temp
                suma2 = suma2 + temp2
                area_foot = area_foot + sx*sy
                ncp=ncp+1
            if (m_val_acum[l][0]>dev_cost*area):  #genero una matriz con los valores acumulados en cada bloque mas la cantidad de bloques de columna
                temp_1= m_val_acum[l][0]-dev_cost*area
                suma_1= suma_1+temp_1
                m_foot[l][0]= suma_1
                nc=nc+1
                m_columna[l][0] = nc
            elif (m_val_acum[l][0] <= dev_cost*area):
                m_foot[l][0] = 0
                m_columna[l][0] = 0
                
                          
       
    if (q in UCL):
        UCL[q].apppend(suma)
        NCP[q].append(ncp)
        UCLton[q].append(suma2)
        Area_foot[q]=[area_foot]
        
    else:
        UCL[q]=[suma]
        NCP[q]=[ncp]
        UCLton[q]= [suma2]
        Area_foot[q]=[area_foot]
                
vpiso=open(project_folder+'\_Footprint_modelo_.xls','w')
vpiso.write("nivel"+"\tZ"+"\tTons"+"\tValor_acum_col"+"\tArea")
for i in UCL.keys(): # fijarme que esta i no se intercepte con las de arriba
    vpiso.write("\n"+str(i))
    vpiso.write("\t"+str(zmin+(i-1)*sz))
    vpiso.write("\t"+str(UCLton[i][0]))
    vpiso.write("\t"+str(UCL[i][0]))
    vpiso.write("\t"+str(Area_foot[i][0]))
vpiso.close() """"""

In [21]:
print m_val

[[-31720.]
 [-31720.]
 [-31720.]
 ..., 
 [     0.]
 [     0.]
 [     0.]]


In [10]:
modelo.to_csv('Simulacion01-sepporcoma.csv', sep=',', header=0, index_col=None)